# Ultralytics Solutions

Ultralytics Solutions ofrece aplicaciones de vanguardia de los modelos YOLO, brindando soluciones prácticas como conteo de objetos, desenfoque y sistemas de seguridad, mejorando la eficiencia y precisión en diversas industrias. Descubre el poder de YOLO11 para implementaciones prácticas y significativas.


En esta práctica vamos a implementar dos tareas relacionadas con la detección de objetos:

- Seguimiento de objetos
- Conteo de objetos

Empezaremos haciendo detección en un video, luego haremos seguimiento de los objetos detectados y finalmente contaremos los objetos que pasan por una línea en el video.

In [ ]:
!pip install -q ultralytics

In [ ]:
from ultralytics import YOLO

#Cargar el modelo pre-entrenado
model = YOLO("yolo11m.pt")

El modelo YOLO11 ya es capaz de detectar maletas, ya que esta clase está incluida en el conjunto de datos COCO, por lo que orientaremos la práctica a la detección de maletas en un video.

In [ ]:
from google.colab import files

uploaded = files.upload() #Selecciona el archivo de video (Solo 1)

In [ ]:
video_path = list(uploaded.keys())[0]

In [ ]:
suitcase_class = 28 #Clase de la maleta en el dataset de COCO

In [ ]:
import cv2

cap = cv2.VideoCapture(video_path)

assert cap.isOpened(), f"Failed to open {video_path}"

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

#Creamos un video de salida para guardar el video procesado
output_path = "output.mp4"

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    #Procesar el frame
    results = model.predict(frame, classes=[suitcase_class]) #Solo detectar maletas

    #Dibujar las detecciones
    annotated_frame = results[0].plot()

    writer.write(annotated_frame)
    cv2.waitKey(1)

cap.release()
writer.release()

**Descarga el video para visualizar los resultados de la práctica:**

## Seguimiento de objetos

El tracking de objetos es una tarea que consiste en seguir un objeto a través de múltiples fotogramas en un video.

El seguimiento de objetos me permite obtener la trayectoria de un objeto en un video, lo que puede ser útil para tareas como el conteo de objetos, la detección de comportamientos anómalos, la predicción de trayectorias, entre otros.


Veamos como se implementa el seguimiento de objetos en un video con YOLO11.

In [ ]:
cap = cv2.VideoCapture(video_path)

assert cap.isOpened(), f"Failed to open {video_path}"

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

#Creamos un video de salida para guardar el video procesado
output_path = "output_track.mp4"

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    #Procesar el frame
    results = model.track(
        frame,
        persist=True, #Seguir las detecciones entre frames
        classes=[suitcase_class], #Solo detectar maletas
    )

    #Dibujar las detecciones
    annotated_frame = results[0].plot()

    writer.write(annotated_frame)
    cv2.waitKey(1)

cap.release()
writer.release()

Puedes observar que solo es necesario modificar la función predict() a track() para implementar el seguimiento de objetos.

En el video observarás que las maletas detectadas ahora incluyen un ID que las identifica a lo largo del video. Si el seguimiento funciona correctamente, las maletas deberían tener el mismo ID en todos los fotogramas en los que aparecen.

## Conteo de objetos

El conteo de objetos nos permitirá saber cuántos objetos de un tipo específico hay en una determinada región de la imagen o cuántos objetos han pasado por una región de interés.

Esta tarea es muy útil en aplicaciones de seguridad, control de tráfico, monitoreo de multitudes, entre otros.

Ultralytics facilita la implementación de esta tarea con YOLO11 y su módulo de soluciones.

Veamos como implementar el conteo de objetos con Ultralytics Solutions.

Primero veamos cuanto es el ancho y alto del video que hemos estado utilizando.

In [ ]:
print(f"Ancho: {width}, Alto: {height}")

Colocaremos una linea en la parte derecha del video, 800 pixeles antes del borde derecho, para contar las maletas que pasan por esa línea.

**Los valores escogidos para la linea son arbitrarios, puedes cambiarlos según tus necesidades.**

Vamos a comprobar la posición de la línea en el video.

In [ ]:
from matplotlib import pyplot as plt

line_pt1 = (width - 200, 100)
line_pt2 = (width - 200, height - 100)

video = cv2.VideoCapture(video_path)

assert video.isOpened(), f"Failed to open {video_path}"

ret, frame = video.read()

cv2.line(frame, line_pt1, line_pt2, (0, 255, 0), 2)

plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

cap.release()

Puedes ver que en el código anterior la linea se dibuja en el video y se encuentra en una posición fija por donde pasan las maletas.

Vamos a crear el objeto necesario para contar las maletas que pasan por la línea.

In [ ]:
from ultralytics import solutions

counter = solutions.ObjectCounter( #Objeto para contar objetos
    classes=[suitcase_class], #Clase a contar, puede ser una lista de clases
    region=[line_pt1, line_pt2], #Region de interes en este caso dos puntos que forman una linea
    show=True, #Mostrar el video en tiempo real
)

cap = cv2.VideoCapture(video_path)

assert cap.isOpened(), f"Failed to open {video_path}"

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

#Creamos un video de salida para guardar el video procesado
output_path = "output_counter.mp4"

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    #Procesar el frame
    results = model.track(
        frame,
        persist=True, #Seguir las detecciones entre frames
        classes=[suitcase_class], #Solo detectar maletas
    )

    annotated_frame = results[0].plot()
    
    #Contar las maletas
    annotated_frame = counter.count(annotated_frame)

    writer.write(annotated_frame)
    cv2.waitKey(1)

cap.release()
writer.release()

Si todo funciona correctamente, deberías ver un contador en la esquina superior izquierda del video que indica cuántas maletas han pasado por la línea.

## Ejercicio

Realiza el conteo de maletas en el segundo video proporcionado.